In [ ]:
!pip install cmake dlib face_recognition numpy

: 

# Libraries

In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

# Socket

In [ ]:
import socket

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 5000))

data = "Gesture: SwipeRight"
client.sendall(data.encode('utf-8'))
client.close()

In [ ]:
pwd

# Path for test images

In [ ]:
path = 'User Images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)

# Extracting names from the List

In [ ]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

# Converting the Images into Encodings

In [ ]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown = findEncodings(images)
print('Encoding Complete')

# Camera  Access and Capturing the Face

In [ ]:
import cv2
import face_recognition
import numpy as np

cap = cv2.VideoCapture(0)
frame_resizing = 0.25

while True:
    success, img = cap.read()
    if not success:
        print("Error: Failed to capture frame.")
        break
    
    imgS = cv2.resize(img, (0, 0), fx=frame_resizing, fy=frame_resizing)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        print('matches', matches)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
        else:
            name = "UNKNOWN"

        # Adjust face location to original image scale
        faceLoc = np.array(faceLoc)
        faceLoc = faceLoc / frame_resizing
        faceLoc = faceLoc.astype(int)
        y1, x2, y2, x1 = faceLoc[0], faceLoc[1], faceLoc[2], faceLoc[3]

        # Draw rectangle and label
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Webcam', img)
    key = cv2.waitKey(1)
    if key == 32:  # Exit when space bar is pressed
        break

cap.release()
cv2.destroyAllWindows()